<a href="https://colab.research.google.com/github/krmonline/Sentiment-Analysis-in-Social-Networks/blob/master/Facebook_Page_Access.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install facebook-sdk

In [0]:
import facebook
import json
import pandas as pd

In [0]:
graph = facebook.GraphAPI(access_token="EAAHqDpf4q4ABAJTIu4ZBy9rHnJZCPQSTmCZANbqW8E3UVV2svK0lkfItoy3If6FZAN4wdw12j4mAeSeFBHw7ZADjhxLS3ZAJ7wzSt7RDu6QkaXKkeILeECU7k1iRUnZAjqbVGUgabJXGeSHhs9ED3iTpRdyik5Xs3gEqRI0fUHfcBW0ZCVyVduuPozNIFO0uXCeCHHLBpePsgQZDZD")

In [0]:
loop_count = 0
df_comments = pd.DataFrame(columns=['id','from','message'])
df = pd.DataFrame(columns=['id','created_time','message','comments'])
js = graph.get_object(id='me/posts',fields='message,story,created_time,id,comments')
while('paging' in js):
  loop_count = loop_count+1
  print(loop_count,end=" ")
  if 'next' in js['paging']:
    arr_page = {}
    for kv in js['paging']['next'].split("?")[1].split("&"):
      k,v = kv.split("=")
      arr_page[k] = v
    until = arr_page['until']
    js = graph.get_object(id='me/posts',fields='message,story,created_time,id,comments',until=until)
    dftmp = pd.DataFrame(js['data'],columns=['id','created_time','message','comments'])

    df = pd.concat([df,dftmp],ignore_index=True)
df

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 

,id,created_time,message,comments
0,3249440901753063_3058965534133935,2020-03-04T04:30:08+0000,NaN,NaN
1,3249440901753063_3057913487572473,2020-03-03T16:59:10+0000,โตมากับฮิโนโทริ,NaN
2,3249440901753063_3057789764251512,2020-03-03T15:56:50+0000,140,NaN
3,3249440901753063_3051348831562272,2020-02-29T19:31:05+0000,NaN,NaN
4,3249440901753063_3049868871710268,2020-02-29T04:58:35+0000,NaN,NaN
...,...,...,...,...
500,3249440901753063_224727733089,2010-01-02T12:58:12+0000,อีก 120 กม ถึงสีคิ้ว วิ่งที่ 85 km/hr,NaN
501,3249440901753063_396722645127,2010-01-02T12:26:24+0000,ถึง เมืองพล แล้ว,NaN
502,3249440901753063_233413932730,2010-01-02T12:14:19+0000,ตอนนี้อยู่บ้านไผ่,NaN
503,3249440901753063_100112503352601,2010-01-01T09:25:35+0000,NaN,NaN


In [0]:
def getMIDfromDF(x):
  return x.split('_')[1]

In [0]:
df['mid'] = df.id.apply(getMIDfromDF)

In [0]:
df

,id,created_time,message,comments,mid
0,3249440901753063_3058965534133935,2020-03-04T04:30:08+0000,NaN,NaN,3058965534133935
1,3249440901753063_3057913487572473,2020-03-03T16:59:10+0000,โตมากับฮิโนโทริ,NaN,3057913487572473
2,3249440901753063_3057789764251512,2020-03-03T15:56:50+0000,140,NaN,3057789764251512
3,3249440901753063_3051348831562272,2020-02-29T19:31:05+0000,NaN,NaN,3051348831562272
4,3249440901753063_3049868871710268,2020-02-29T04:58:35+0000,NaN,NaN,3049868871710268
...,...,...,...,...,...
500,3249440901753063_224727733089,2010-01-02T12:58:12+0000,อีก 120 กม ถึงสีคิ้ว วิ่งที่ 85 km/hr,NaN,224727733089
501,3249440901753063_396722645127,2010-01-02T12:26:24+0000,ถึง เมืองพล แล้ว,NaN,396722645127
502,3249440901753063_233413932730,2010-01-02T12:14:19+0000,ตอนนี้อยู่บ้านไผ่,NaN,233413932730
503,3249440901753063_100112503352601,2010-01-01T09:25:35+0000,NaN,NaN,100112503352601


In [0]:
if len(df[df.comments.isna() == False]) > 0:
  for i in  df[df.comments.isna() == False].comments.values:
    text = i['data']
    dftmp_comments = pd.DataFrame(i['data'],columns=['id','from','message'])
    df_comments = pd.concat([df_comments,dftmp_comments],ignore_index=True)

In [0]:
df_comments

,id,from,message
0,2164745816889249_2164799203550577,"{'name': 'Chakrit Phain', 'id': '3249440901753...",555 ครับพี่
1,2164745816889249_2165217246842106,"{'name': 'Chakrit Phain', 'id': '3249440901753...",อืออ
2,2164504823580015_2167603746603456,"{'name': 'Chakrit Phain', 'id': '3249440901753...",โหหห ขอบคุณครับ Kankawin Kowsrihawat
3,2057217860975379_2057224364308062,"{'name': 'Chakrit Phain', 'id': '3249440901753...",รายยย
4,2057217860975379_2057226877641144,"{'name': 'Chakrit Phain', 'id': '3249440901753...",ได้เรยครับบ 😆
...,...,...,...
209,171162409580943_267827,"{'name': 'Chakrit Phain', 'id': '3249440901753...",ฮิ
210,170958396268011_267835,"{'name': 'Chakrit Phain', 'id': '3249440901753...",ชอบภาพนี้มากครับ
211,170958396268011_292639,"{'name': 'Chakrit Phain', 'id': '3249440901753...",แท็ก ภาพพี่อารี ฮิฮิ
212,170942186269632_271772,"{'name': 'Chakrit Phain', 'id': '3249440901753...",ยังไม่แตะเน้อ


In [0]:
def getMID(x):
  return x.split('_')[0]
def getCID(x):
  return x.split('_')[1]
def getUserComment(x):
  return x['name']

In [0]:
df_comments['mid'] = df_comments.id.apply(getMID)
df_comments['cid'] = df_comments.id.apply(getCID)
df_comments['cname'] = df_comments['from'].apply(getUserComment)

In [0]:
df_comments.head()

,id,from,message,mid,cid,cname
0,2164745816889249_2164799203550577,"{'name': 'Chakrit Phain', 'id': '3249440901753...",555 ครับพี่,2164745816889249,2164799203550577,Chakrit Phain
1,2164745816889249_2165217246842106,"{'name': 'Chakrit Phain', 'id': '3249440901753...",อืออ,2164745816889249,2165217246842106,Chakrit Phain
2,2164504823580015_2167603746603456,"{'name': 'Chakrit Phain', 'id': '3249440901753...",โหหห ขอบคุณครับ Kankawin Kowsrihawat,2164504823580015,2167603746603456,Chakrit Phain
3,2057217860975379_2057224364308062,"{'name': 'Chakrit Phain', 'id': '3249440901753...",รายยย,2057217860975379,2057224364308062,Chakrit Phain
4,2057217860975379_2057226877641144,"{'name': 'Chakrit Phain', 'id': '3249440901753...",ได้เรยครับบ 😆,2057217860975379,2057226877641144,Chakrit Phain


In [0]:
df_result = pd.merge(df_comments,
                 df[['id','created_time','message','mid']],
                 on='mid', 
                 how='left')

In [0]:
df_result.drop(['from'],axis=1).to_csv('./result.csv')